In [2]:
import os
import sys
import time
import pprint

from eth_account import Account
from eth_account.signers.local import LocalAccount
import web3
from web3 import Web3, EthereumTesterProvider
from web3.middleware import construct_sign_and_send_raw_middleware
import solcx

# Set-up Ganache & Accounts

In [3]:
ganache_url = "http://127.0.0.1:8545"
w3 = web3.Web3(web3.Web3.HTTPProvider(ganache_url))

In [4]:
account_addr = "0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1"
private_key = "0x4f3edf983ac636a65a842ce7c78d9aa706d3b113bce9c46f30d7d21715b23b1d"

account: LocalAccount = Account.from_key(private_key)
w3.middleware_onion.add(construct_sign_and_send_raw_middleware(account))

print(f"Your hot wallet address is {account.address}")

Your hot wallet address is 0x6531e2613bbbEEcd898356F7b9bEfBaEfd42804B


# Functions

In [47]:
def compile_source_file(file_path):
    with open(file_path, 'r') as f:
        source = f.read()
    return solcx.compile_source(source, output_values=["abi", "bin", "bin-runtime"])


def deploy_contract(w3, account, contract_interface):
    func = w3.eth.contract(
        abi=contract_interface["abi"],
        bytecode=contract_interface["bin"]).constructor()
    #
    gas_estimate = func.estimate_gas()
    print("gas_estimate=", gas_estimate) 
    #
    tx_hash = func.transact({"from": account})
    address = w3.eth.get_transaction_receipt(tx_hash)["contractAddress"]
    return address

def gas_estimate_for_func(w3, func):
    # Get current gas price in GWEI from https://etherscan.io/gastracker.
    gas_price = get_curr_gas_gwei()
    print("Current gas price [GWEI]=", gas_price)
    gas_estimate = func.estimate_gas()
    print("Gas estimate for func [units]=", gas_estimate)
    gas_to_usd(gas_estimate, gas_price)
    
    
def gas_to_usd(gas_estimate, gas_price):
    # Convert price to WEI.
    gas_price_wei = gas_price * 1000000000
    # Calculate function cost in WEI.
    gas_cost = gas_estimate * gas_price_wei
    # Calculate function cost in ETH.
    gas_cost_eth = w3.fromWei(gas_cost, "ether")
    print("Gas price for func [ETH]=", gas_cost_eth)
    #
    eth_price_USD = 1523
    gas_cost_USD = gas_cost_eth * eth_price_USD
    print("Gas price for func [USD]=", gas_cost_USD)
    
    
def get_curr_gas_gwei():
    return 18
    

# Experiments

In the experimients we'll concentracte on the price of iterating the array of buy/ sell orders, since we're doing it twice in our code and this is considered as a coslty operation. Let's measure how the gas amount will scale with increasing of the numbers of orders.

We will use only Buy Orders here since there is no big computational difference which order to iterate + we don't have ERC20 tokens to send to simulate Sell Order.

In [7]:
compiled = compile_source_file("contracts/DaoSwap.sol")
contract_interface = compiled["<stdin>:DaoSwap"]

In [12]:
contractName = "DaoSwap";
tokens = ["0xfb6115445Bff7b52FeB98650C87f44907E58f802"];
swapPeriodInSecs = 300;
swapRandomizationInSecs = 5;
feesAsPct = 5;
priceMode = 3;
priceOracle = "0x547a514d5e3769680Ce22B2361c10Ea13619e8a9";
swapMode = 1;

func=w3.eth.contract(
    abi=contract_interface["abi"],
    bytecode=contract_interface["bin"]).constructor(
        contractName, 
        tokens,
        swapPeriodInSecs,
        swapRandomizationInSecs,
        feesAsPct,
        priceMode,
        priceOracle,
        swapMode)

Estimate the price of deployment.

In [23]:
# Get the price for deployment.
gas_estimate_for_func(w3, func)

Current gas price [GWEI]= 18
Gas estimate for func [units]= 2701653
Gas price for func [ETH]= 0.048629754
Gas price for func [USD]= 74.063115342


Deploy the contract.

In [14]:
tx_hash = func.transact({"from": account_addr})
address = w3.eth.get_transaction_receipt(tx_hash)["contractAddress"]

From ganache: gas usage 2778140

Get DaoSwap contract instance.

In [16]:
contract = w3.eth.contract(address=address, abi=contract_interface["abi"])

Construct Buy Orders arguments.

In [17]:
token_to_buy="0xfb6115445Bff7b52FeB98650C87f44907E58f802"
amount_of_token_18dec=1000000000000000000
limit_price=54430000000000000
deposit_addr="0x5B38Da6a701c568545dCfcB03FcB875f56beddC4"

In [18]:
buy_order = contract.functions.buyOrder(token_to_buy,
                           amount_of_token_18dec,
                           limit_price,
                           deposit_addr).transact(
                    {"from": account_addr, "value": 54430000000000000})

In [23]:
# 183893 gas
w3.eth.wait_for_transaction_receipt(buy_order)

AttributeDict({'transactionHash': HexBytes('0x1922fc32053543102696bd809160ee187a71b8b1511e5bec667a5cd20183f8ae'),
 'transactionIndex': 0,
 'blockNumber': 3,
 'blockHash': HexBytes('0x3ff94b4150a47eafe0d50c6740ad43533cda6b501a1ae74b270c2b677f241bed'),
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0x5b1869D9A4C187F2EAa108f3062412ecf0526b24',
 'cumulativeGasUsed': 183893,
 'gasUsed': 183893,
 'contractAddress': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status':

Check if the order was added - get the current array of Orders.

In [24]:
contract.functions.getOrders().call({"from": account_addr})

[(True,
  False,
  '0xfb6115445Bff7b52FeB98650C87f44907E58f802',
  '0x0000000000000000000000000000000000000000',
  '0x5B38Da6a701c568545dCfcB03FcB875f56beddC4',
  '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
  1676270695,
  1000000000000000000,
  54430000000000000)]

We have for loops in `getTotals` and `executeProportional` functions. Let's check how amount of consumed gas will increase with the growth of Orders array.

Iterate one order.

In [29]:
price_of_token = 54330000000000000
print_gas_estimate(w3, contract.functions.getTotals(price_of_token))

Current gas price [GWEI]= 18
Gas estimate for func [units]= 31680
Gas price for func [ETH]= 0.00057024
Gas price for func [USD]= 0.86847552


In [31]:
contract.functions.getTotals(price_of_token).call()
# 1000000000000000000 = 1 token with 18 decimals

[1000000000000000000, 0]

Add one more order and check how the gas price will change.

In [32]:
buy_order = contract.functions.buyOrder(token_to_buy,
                           amount_of_token_18dec,
                           limit_price,
                           deposit_addr).transact(
                    {"from": account_addr, "value": 54430000000000000})

In [33]:
contract.functions.getOrders().call({"from": account_addr})

[(True,
  False,
  '0xfb6115445Bff7b52FeB98650C87f44907E58f802',
  '0x0000000000000000000000000000000000000000',
  '0x5B38Da6a701c568545dCfcB03FcB875f56beddC4',
  '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
  1676270695,
  1000000000000000000,
  54430000000000000),
 (True,
  False,
  '0xfb6115445Bff7b52FeB98650C87f44907E58f802',
  '0x0000000000000000000000000000000000000000',
  '0x5B38Da6a701c568545dCfcB03FcB875f56beddC4',
  '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
  1676271496,
  1000000000000000000,
  54430000000000000)]

Iterate two orders.

In [35]:
print_gas_estimate(w3, contract.functions.getTotals(price_of_token))

Current gas price [GWEI]= 18
Gas estimate for func [units]= 39167
Gas price for func [ETH]= 0.000705006
Gas price for func [USD]= 1.073724138


We already have 2 orders, let's add 98 more to get 100 overall. Let's imagine that we're a famous swap and can get 100 orders in 300 seconds!

In [36]:
for i in range(98):
    contract.functions.buyOrder(token_to_buy,
                           amount_of_token_18dec,
                           limit_price,
                           deposit_addr).transact(
                    {"from": account_addr, "value": 54430000000000000})
    

In [38]:
len(contract.functions.getOrders().call({"from": account_addr}))

100

Iterate 100 orders.

In [39]:
print_gas_estimate(w3, contract.functions.getTotals(price_of_token))

Current gas price [GWEI]= 18
Gas estimate for func [units]= 772893
Gas price for func [ETH]= 0.013912074
Gas price for func [USD]= 21.188088702


But `getTotals` is not the only function with the for loop we have! To reproduce the whole swap cycle, we also need to `executeProportional`, which iterates the orders and sends eth/tokens to users.

In [44]:
tokens_to_buy = 102000000000000000000
tokens_to_sell = 0
decimals = 3

execute_prop = contract.functions.executeProportional(
    tokens_to_buy, 
    tokens_to_sell, 
    decimals, 
    price_of_token).call({"from": account_addr})


In [50]:
# Function does not return anything.
execute_prop

[]

I took transaction hash from ganache console.

In [45]:
w3.eth.wait_for_transaction_receipt("0x3b142c88da51d827f2cba8cc5b05dc9a769129ce2fb7fabb00cd039dbe4d2114")

AttributeDict({'transactionHash': HexBytes('0x3b142c88da51d827f2cba8cc5b05dc9a769129ce2fb7fabb00cd039dbe4d2114'),
 'transactionIndex': 0,
 'blockNumber': 101,
 'blockHash': HexBytes('0x1f95b699cbb3bca375bcef6cb0950ac3e59c866acbd52195ebdf6666b5934dc8'),
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xe78A0F7E598Cc8b0Bb87894B0F60dD2a88d6a8Ab',
 'cumulativeGasUsed': 166793,
 'gasUsed': 166793,
 'contractAddress': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status

Also we need to keep in mind that `executeProportional` sends tokens to users by design, but in compiled code the line that sends tokens
`IERC20(orders[i].tokenToBuy).transfer(orders[i].depositAddress, tokenAmount);`
is commented out since we don't have tokens to send.


In [48]:
gas_to_usd(166793, 18)

Gas price for func [ETH]= 0.003002274
Gas price for func [USD]= 4.572463302


**How to improve: combine mapping and array**

From https://www.devtwins.com/blog/understanding-mapping-vs-array-in-solidity:
    
`If you need to be able to iterate over your group of data (such as with a for loop), then use an array.

If you don't need to be able to iterate over your data, and will be able to fetch values based on a known key, then use a mapping.

However, sometimes it is optimal to use both. Since iterating over an array can be an expensive action in Solidity (compared to fetching data from a mapping), and since you may want to be able to store both a value and its key within your smart contract, developers sometimes opt to create an array of keys, which serve as a reference to even more data that can then be retrieved from its associated value inside of a mapping.

Keep in mind that you should never allow an array in Solidity to grow too large, since in theory iterating over a big enough array could end up costing more in gas fees than the value of the transaction is worth (another reason to consider using mapping when possible).`

**About the gas refunds**

After each swap we erase the orders array, so we'll get some gas refunds.

From https://soliditydeveloper.com/design-pattern-solidity-free-up-unused-storage:

You can receive gas refunds for releasing unused storage. In the yellow paper on page 25 'Appendix G. Fee Schedule', you can read the gas costs for each instruction. As you might know, SSTORE will generally create the most costs in your contracts with a significant cost of 20,000 gas per instruction. On the contrary, if you look at R_sclear:

Refund given when the storage value is set to zero from non-zero.

15,000 gas refund means you can actually get 75% of your storing costs back! That is a large amount, do not forget about this. And the solution is simple, just set a value back to 0 once you are sure it will not be used anymore.

